In [1]:
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
alldata={}
models=[]
for i in next(os.walk('.'))[1]:
    try:
        int(i)
        models.append(i)
    except ValueError as ve:
        continue
models.sort()

data=np.zeros((len(models),3))
counter=0
for i in models:
        x = pd.read_csv('{}/exp_details.csv'.format(i),header=None)
        data[counter]= [i,x.shape[0],x[x[2]>0].shape[0]]
        counter+=1

labels =[]
for i in range(data.shape[0]):
    labels.append(str(int(data[i,0])))

fig = plt.figure(figsize=(20,10))
ax = plt.subplot(1,1,1)
ax.bar(data[1:,0],data[1:,1])
ax.bar(data[1:,0]+0.5,data[1:,2],color='r')
ax.set_xticks(data[1:,0])
ax.set_xticklabels(labels[1:])
ax.set_xlabel('Models')

In [2]:
import pandas as pd
import os
from Test_Functions import ReadCSV

## Read all new general feature and set them in csv data set

In [3]:
source = open('features.results.out')
lines = [x.strip() for x in source.readlines()]
lines = ['{},{}{}'.format(lines[i],lines[i+1],'\n') for i in range(0,len(lines),2)]
lines = [i.replace('[','"[') for i in lines]
lines = [i.replace(']',']"') for i in lines]
target = open('semifinal.csv','w')
#Normal ones
#lines = ['experiment,reply_size,layers,tau,optimizer,advantage,max_steps,activation,batch_size,totalsteps,exploration,vanish,gamma,hidden_size,train_repeat,batch_norm,seed,rwrdschem,svision,details\n']+lines
#Ones with Convolution
lines = ['experiment,reply_size,layers,tau,optimizer,advantage,max_steps,activation,batch_size,totalsteps,exploration,vanish,gamma,hidden_size,train_repeat,batch_norm,seed,rwrdschem,svision,nconv,cnnfilter,cnnsize,optimizer_lr,details\n']+lines
target.writelines(lines)
target.close()

## Add Experiment details.
* Train :
    * count
    * Total Reward
    * Total step
    * Total time
    
* Test:
    * count
    * Total Reward
    * Total step
    * Total time

In [4]:
x = pd.read_csv('semifinal.csv')

In [5]:
#Train information
# Episodes count
x['tr_count']=0
# Total Reward (sum over all epsidoes)
x['tr_TR']=0
# Total Time (sum over all episodes)
x['tr_TT']=0
# Total steps (sum over all episodes)
x['tr_TS']=0
#Test Information
# Episodes count
x['ts_count']=0
# Total Reward (sum over all epsidoes)
x['ts_TR']=0
# Total Time (sum over all episodes)
x['ts_TT']=0
# Total steps (sum over all episodes)
x['ts_TS']=0
# percentage of training episodes with positive reward
x['tr_s']=0
# percentage of testing episodes with positive reward
x['ts_s']=0

x['tr_AR'] = 0.0
x['tr_AS'] = 0.0
x['ts_AR'] = 0.0
x['ts_AS'] = 0.0
#training Steps median after exploration vanish.
x['tr_SMD']=0.0
#training Steps mean after exploration vanish.
x['tr_SM']=0.0
#training Reward median after exploration vanish.
x['tr_RMD']=0.0
#training Reward mean after exploration vanish.
x['tr_RM']=0.0
#testing Steps median after exploration vanish.
x['ts_SMD']=0.0
#testing Steps mean after exploration vanish.
x['ts_SM']=0.0
#testing Reward median after exploration vanish.
x['ts_RMD']=0.0
#testing Reward mean after exploration vanish.
x['ts_RM']=0.0

In [6]:
for i in x.experiment.as_matrix():
    train,test,splitpoint = ReadCSV(i,x[x.experiment==i].vanish.as_matrix())
    
    indx = x[x['experiment']==i].index[0]
    x.set_value(indx, 'tr_count',train.shape[0])
    x.set_value(indx, 'tr_TR',train[2].sum())
    x.set_value(indx, 'tr_TT',train[3].sum())
    x.set_value(indx, 'tr_TS',train[1].sum())
    
    x.set_value(indx, 'ts_count',test.shape[0])
    x.set_value(indx, 'ts_TR',test[2].sum())
    x.set_value(indx, 'ts_TT',test[3].sum())
    x.set_value(indx, 'ts_TS',test[1].sum())
    tr_tmp = train[2]
    ts_tmp = test[2]
    x.set_value(indx,['tr_s'],tr_tmp[tr_tmp>0].count()/tr_tmp.count(),)
    x.set_value(indx,['ts_s'],ts_tmp[ts_tmp>0].count()/ts_tmp.count())
    
    train = train[train[0]>=splitpoint]
    test = test[test[0]>=(splitpoint/10)]
    
    x.set_value(indx,'tr_SMD',train[1].median())
    x.set_value(indx,'tr_SM',train[1].mean())
    x.set_value(indx,'tr_RMD',train[2].median())
    x.set_value(indx,'tr_RM',train[2].mean())

    x.set_value(indx,'ts_SMD',test[1].median())
    x.set_value(indx,'ts_SM',test[1].mean())
    x.set_value(indx,'ts_RMD',test[2].median())
    x.set_value(indx,'ts_RM',test[2].mean())
    #train = train[2]
    #test = test[2]

    print(i)

796
795
794
798
793
792
797
806
821
817
808
818
822
823
820
802
801
799
803
804
813
814
800
810
809
819
847
848
824
853
830
815
812
825
854
850
852
835
849
816
844
828
851
811
846
826
842
833
829
805
838
827
843
831
807
837
841
840
839
845
832
836
834
858
861
856
859
860
857
855
903
896
902
879
897
868
884
888
895
886
898
863
887
862
873
901
891
869
885
870
880
881
866
883
871
875
890
892
872
878
882
874
867
893
865
877
900
899
894
864
889
876


In [7]:
x['tr_AR'] = x['tr_TR']/x['tr_count']
x['tr_AS'] = x['tr_TS']/x['tr_count']
x['ts_AR'] = x['ts_TR']/x['ts_count']
x['ts_AS'] = x['ts_TS']/x['ts_count']

In [8]:
x.to_csv('semifinal.csv',index=False)

In [9]:
x.sort_values('experiment')[['experiment','details']]

,experiment,details
5,792,CNN subordinate only
4,793,CNN subordinate only
2,794,CNN subordinate only
1,795,CNN subordinate only
0,796,CNN subordinate only
6,797,CNN subordinate only
3,798,CNN subordinate only
17,799,CNN subordinate only
22,800,CNN subordinate only
16,801,CNN subordinate only


## Merge current results with Final.csv and sort the result.

In [10]:
Final = pd.read_csv('Final.csv')

In [11]:
print('new data shape:{},old data shape:{}'.format(x.shape,Final.shape))
Final = pd.concat([Final,x])
print('new Full shape:',Final.shape)
Final.sort_values('experiment',inplace=True)
Final.to_csv('Final.csv',index=False)

new data shape:(112, 46),old data shape:(784, 42)
new Full shape: (896, 46)


In [12]:
pd.options.display.max_columns=80

In [14]:
Final[Final.experiment==900]

,activation,advantage,batch_norm,batch_size,cnnfilter,cnnsize,details,experiment,exploration,gamma,hidden_size,layers,max_steps,nconv,optimizer,optimizer_lr,reply_size,rwrdschem,seed,svision,tau,totalsteps,tr_AR,tr_AS,tr_RM,tr_RMD,tr_SM,tr_SMD,tr_TR,tr_TS,tr_TT,tr_count,tr_s,train_repeat,ts_AR,ts_AS,ts_RM,ts_RMD,ts_SM,ts_SMD,ts_TR,ts_TS,ts_TT,ts_count,ts_s,vanish
106,relu,avg,False,32,"[32, 16]","[3, 3]",CNN subordinate only,900,0.1,0.99,100,1,1000,2.0,adam,0.001,100000,"[-10.0, 1000.0, -0.1]",6654,180,0.001,2000000,400.833918,8.163715,427.292602,-0.5,8.040071,8.0,98199099,2000004,139112,244987,0.412985,1,426.932324,7.912364,455.976077,-0.5,7.763344,7.0,10459415,193845,1668,24499,0.437569,0.75
